In [38]:
import tensorflow as tf
import cv2
import os
from tensorflow.keras import layers,models
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [31]:
df=pd.read_csv("chinese_mnist.csv")
df.head(5)

,suite_id,sample_id,code,value,character
0,1,1,10,9,九
1,1,10,10,9,九
2,1,2,10,9,九
3,1,3,10,9,九
4,1,4,10,9,九


In [46]:
class_names=df.character.unique()
labels=[]
data=np.zeros((df.shape[0],64,64,1))
for i in range(df.shape[0]):
    path="data\data\input_"+str(df.loc[i,"suite_id"])+"_"+str(df.loc[i,"sample_id"])+"_"+str(df.loc[i,"code"])+".jpg"
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
    img=img/255.0
    img=img.reshape(64,64,1)
    labels.append(df.loc[i,"value"])
    data[i,:,:]=img

In [33]:
x_train,x_test,y_train,y_test=train_test_split(data,labels,test_size=0.2,random_state=42)

In [45]:
network=models.Sequential()
network.add(layers.Conv2D(32,(5,5),activation='relu', input_shape=(64,64,1)))
network.add(layers.Conv2D(64,(3,3),activation='relu'))
network.add(layers.MaxPooling2D((3,3)))

network.add(layers.Conv2D(64,(3,3),activation='relu'))
network.add(layers.Conv2D(32,(3,3),activation='relu'))
network.add(layers.MaxPooling2D((3,3)))
network.add(layers.Flatten())
network.add(layers.Dropout(0.25))
network.add(layers.Dense(80,activation='relu'))
network.add(layers.Dense(50,activation='relu'))
network.add(layers.Dense(15,activation='softmax'))
network.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 60, 60, 32)        832       
                                                                 
 conv2d_12 (Conv2D)          (None, 58, 58, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 19, 19, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 17, 17, 64)        36928     
                                                                 
 conv2d_14 (Conv2D)          (None, 15, 15, 32)        18464     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 5, 5, 32)          0         
 g2D)                                                 